In [21]:
"""
Just a demo of using direct calls with OpenAI util to parse and use tools
In this example there simple numeric operations
Simpler than v1 (coming more from openai docs)
"""

'\nJust a demo of using direct calls with OpenAI util to parse and use tools\nIn this example there simple numeric operations\nSimpler than v1 (coming more from openai docs)\n'

In [22]:
import os
import openai
from openai import OpenAI
import json

import os
from dotenv import load_dotenv

In [23]:
api_key = os.getenv("OPENAI_API_KEY")


# Define tools 

In [46]:
tools = [
    {
        "type": "function",
        "name": "add_numbers",
        "description": "Add (sums) two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "First number to sum / add"},
                "b": {"type": "number", "description": "Second number to sum / add"},
            },
            "required": ["a", "b"],  
            "additionalProperties": False          
        }
    }]
   

def add_numbers(a, b):
    return {"result": a + b}

tool_functions = {
    "add_numbers": add_numbers,
}

def function_call(func_name : str, args : dict):
    if func_name in tool_functions:
        func_obj = tool_functions[func_name]
        return func_obj(**args)
    else:
        return {"error": f"Function {func_name} not found"}

def handle_function_call(tool_call):


    tool_name = tool_call.name
    arguments = json.loads(tool_call.arguments)

    result = function_call(tool_name, arguments)

    res = {
        "role": "function_call_output",
        "output": result['result'],
        "call_id": tool_call.id
    }

    return res 

def handle_function_calls(tool_calls):
    res = []
    for tool_call in tool_calls:
        if tool_call.type == "function_call":
            tool_result = handle_function_call(tool_call)
            
            res.append(tool_result)
        else:
            continue
        
    return res






In [25]:
# Run model

In [43]:
client = OpenAI(api_key=api_key)

system_prompt = """
    You are a helpful assistant that uses tools to conduct simple numeric operations
    You will be given a task to perform and you will use the tools to perform the task
    You can only do additions now. Do not try to do anything else. Do not make up results.
"""

input_messages =  [
        {"role" : "system", "content" : system_prompt},
        {"role" : "user", "content" : "What is the sum of 5 and 7?"}
    ]

response = client.responses.create(
    model = "gpt-4.1",
    input = input_messages,
    tools = tools
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"a":5,"b":7}', call_id='call_g4KtInEFt1B41nU8h4xKsJkY', name='add_numbers', type='function_call', id='fc_688a6feb753c819890393a80fcacfad20302f178daca30e5', status='completed')]


In [39]:
response.output[0].to_json()

'{\n  "arguments": "{\\"a\\":5,\\"b\\":7}",\n  "call_id": "call_SeHJ8U7EViTjJZzG71KUgzrA",\n  "name": "add_numbers",\n  "type": "function_call",\n  "id": "fc_688a6e0a01e081999282b32dde783d94094a0d720f61b6a7",\n  "status": "completed"\n}'

In [49]:
# Handling function calls and running model again

tool_call_res = handle_function_calls(response.output)

new_input_messages = input_messages.copy()
new_input_messages.append(response.output[0])
#new_input_messages.extend(tool_call_res)

response2 = client.responses.create(
    model = "gpt-4.1",
    input = new_input_messages,
    tools = tools
)

print(response2.output_text)

BadRequestError: Error code: 400 - {'error': {'message': 'No tool output found for function call call_g4KtInEFt1B41nU8h4xKsJkY.', 'type': 'invalid_request_error', 'param': 'input', 'code': None}}

In [45]:
tool_call_res

[{'type': 'function_call_output',
  'output': 12,
  'call_id': 'fc_688a6feb753c819890393a80fcacfad20302f178daca30e5'}]

In [ ]:
handle_function_calls(response.output)

Tool called: add_numbers


[{'type': 'function_call_output',
  'output': '{"result": 12}',
  'call_id': 'fc_688a6cfb98c48198b818d70ff03846ad0abbab1559a9267d'}]